# COUNT-MIN sketch task

In [1]:
import numpy as np
import pandas as pd
import mmh3
import math
from utils import read_data
import operator
import time

## Read the dataset and keep only the flows that contain the infected host

In [2]:
infected_host = '147.32.84.165'

data = read_data('datasets/CTU-Malware-Capture-Botnet-54')
# data.to_pickle('./data.pkl')
# data = pd.read_pickle('./data.pkl')
infected_dataset = data.loc[(data['src_ip'] == infected_host) | (data['dst_ip'] == infected_host)]
print('Rows with infected host: {}'.format(infected_dataset.shape[0]))

Rows with infected host: 21760


## Helper functions

### Finds the frequency and the number of flows for each IP in the infected dataset

In [3]:
def compute_most_frequent(infected_dataset):
    connections = {}
    
    # compute the number of flows for each ip
    for index, row in infected_dataset.iterrows():
        src = row['src_ip']
        dst = row['dst_ip']
        if src == infected_host:
            if dst in connections:
                connections[dst] += 1
            else:
                connections[dst] = 1
        elif dst == infected_host:
            if src in connections:
                connections[src] += 1
            else:
                connections[src] = 1
    # sor the results
    sorted_connections = sorted(connections.items(), key=operator.itemgetter(1), reverse=True)
    total_connections = len(infected_dataset)
    
    # create a dataframe with the frequency ans the number of connections for each ip
    connection_df = pd.DataFrame(sorted_connections, columns=['IP', 'num_of_connections'])
    connection_df['frequency'] = round(100 * connection_df['num_of_connections'] / total_connections, 2)
    return connection_df

### Finds the difference in the frequence for the top 10 IPs , between the true sequence and the one obtained from Reservoir sampling

In [4]:
def find_differences(normal_top, sampled):
    diff = 0
    for index, row in normal_top.iterrows():
        if row['IP'] in sampled['IP'].values:
            normal_freq = row['frequency']
            sampled_freq = sampled.loc[sampled['IP'] == row['IP']].iloc[0]['frequency']
            diff += abs(normal_freq - sampled_freq)
        else:
            diff += row['frequency']
    return diff

### Finds the difference in the frequence for the top 10 IPs , between the true sequence and the one obtained from Reservoir sampling

In [5]:
normal_top = compute_most_frequent(infected_dataset)[:10]
normal_top_ips = normal_top['IP'].tolist()

## Class which contains all the functionallity of the COUNT-MIN sketch

In [6]:
class CountMinSketch(object):
    def __init__(self, w, d):
        self.d = d
        self.w = w
        self.cm_array =  np.zeros((d, w), dtype=np.int32)
    
    # Add a new flow to the array
    def add(self, key):
        for i in range(self.d):
            index = mmh3.hash(key, i) % self.w
            self.cm_array[i][index] += 1
            
    # Find the value of a flow
    def point(self, key):
        min_value = math.inf
        for i in range(self.d):
            index = mmh3.hash(key, i) % self.w
            if self.cm_array[i][index]< min_value:
                min_value = self.cm_array[i][index]
        return min_value
        

## Perform the COUNT-MIN sketch and find the top 10 IPs for different values of w and d

In [7]:
# compute the frequnt ips for different values of d and w
# we initially define epsilon and delta values we want, and we use these values to compute w and d 
for epsilon in [0.0001, 0.001, 0.005, 0.01, 0.1]:
    for delta in [0.0001, 0.001, 0.005, 0.01, 0.1]:  
        
        # calculate the w, d
        w = int(round(math.e/epsilon))
        d = int(round(math.log(1/delta)))
        space = w*d
        print('---------- w={}, d={}, space={} ----------\n'.format(w, d, space))

        # construct the matrix with the correct dimensions
        count_min_matrix = CountMinSketch(w, d)

        # add each ip to the matrix
        ips = []
        for index, row in infected_dataset.iterrows():
            src = row['src_ip']
            dst = row['dst_ip']
            if src == infected_host:
                ips += [dst]
            elif dst == infected_host:
                ips += [src]
        
        for ip in ips:
            count_min_matrix.add(ip)
            
        # find frequency and store it
        count_min = {}
        for ip in set(ips):
            count_min[ip] = count_min_matrix.point(ip)
        
        total_connections = infected_dataset.shape[0]
        
        # sort them according to their value to find the 10 most frequent ones
        sorted_count_min = sorted(count_min.items(), key=operator.itemgetter(1), reverse = True)
        min_sketch = pd.DataFrame(sorted_count_min, columns=['IP', 'num_of_connections'])
        min_sketch['frequency'] = round(100 * min_sketch['num_of_connections'] / total_connections, 2)
        print(min_sketch[:10],'\n')
        min_sketch_ips = min_sketch[:10]['IP'].tolist()
        print('\nDifferent IPs: {}'.format(len(set(normal_top_ips) - set(min_sketch_ips))))
        diff = find_differences(normal_top, min_sketch)
        print("Frequency difference: %0.3f\n" %diff)

---------- w=27183, d=9, space=244647 ----------

                IP  num_of_connections  frequency
0      147.32.80.9                9774      44.92
1   184.173.217.40                2287      10.51
2  212.117.171.138                1725       7.93
3     65.55.37.104                 391       1.80
4    65.54.188.110                 198       0.91
5    94.63.149.150                 157       0.72
6     74.125.39.27                 143       0.66
7    205.188.103.1                 127       0.58
8     65.55.92.152                 120       0.55
9     74.125.93.27                 115       0.53 


Different IPs: 0
Frequency difference: 0.000

---------- w=27183, d=7, space=190281 ----------

                IP  num_of_connections  frequency
0      147.32.80.9                9774      44.92
1   184.173.217.40                2287      10.51
2  212.117.171.138                1725       7.93
3     65.55.37.104                 391       1.80
4    65.54.188.110                 198       0.91
5

                IP  num_of_connections  frequency
0      147.32.80.9                9774      44.92
1   184.173.217.40                2287      10.51
2  212.117.171.138                1727       7.94
3     65.55.37.104                 391       1.80
4    65.54.188.110                 204       0.94
5    94.63.149.150                 160       0.74
6     74.125.39.27                 145       0.67
7    205.188.103.1                 127       0.58
8     65.55.92.152                 120       0.55
9     74.125.93.27                 115       0.53 


Different IPs: 0
Frequency difference: 0.070

---------- w=544, d=2, space=1088 ----------

                IP  num_of_connections  frequency
0      147.32.80.9                9774      44.92
1   184.173.217.40                2287      10.51
2  212.117.171.138                1727       7.94
3     65.55.37.104                 393       1.81
4    65.54.188.110                 205       0.94
5    94.63.149.150                 160       0.74
6    

## Perform the COUNT-MIN sketch for standard w and d values, and measure execution time

In [8]:
# we initially define epsilon and delta values we want, and we use these values to compute w and d 
epsilon = 0.01
delta = 0.005        

# calculate the w, d
w = int(round(math.e/epsilon))
d = int(round(math.log(1/delta)))
space = w*d
print('---------- w={}, d={}, space={} ----------\n'.format(w, d, space))

start = time.time()

# construct the matrix with the correct dimensions
count_min_matrix = CountMinSketch(w, d)

# add each ip to the matrix
ips = []
for index, row in infected_dataset.iterrows():
    src = row['src_ip']
    dst = row['dst_ip']
    if src == infected_host:
        ips += [dst]
    elif dst == infected_host:
        ips += [src]

for ip in ips:
    count_min_matrix.add(ip)

# find frequency and store it
count_min = {}
for ip in set(ips):
    count_min[ip] = count_min_matrix.point(ip)

total_connections = infected_dataset.shape[0]
# sort them according to their value to find the 10 most frequent ones
sorted_count_min = sorted(count_min.items(), key=operator.itemgetter(1), reverse = True)
min_sketch = pd.DataFrame(sorted_count_min, columns=['IP', 'num_of_connections'])
min_sketch['frequency'] = round(100 * min_sketch['num_of_connections'] / total_connections, 2)

# stop time recording
stop = time.time()
print(stop - start)

print(min_sketch[:10])
min_sketch_ips = min_sketch[:10]['IP'].tolist()
print('\nDifferent IPs: {}'.format(len(set(normal_top_ips) - set(min_sketch_ips))))
diff = find_differences(normal_top, min_sketch)
print("Frequency difference: %0.3f\n" %diff)

---------- w=272, d=5, space=1360 ----------

2.1685492992401123
                IP  num_of_connections  frequency
0      147.32.80.9                9780      44.94
1   184.173.217.40                2289      10.52
2  212.117.171.138                1728       7.94
3     65.55.37.104                 398       1.83
4    65.54.188.110                 217       1.00
5    94.63.149.150                 160       0.74
6     74.125.39.27                 145       0.67
7    205.188.103.1                 127       0.58
8     65.55.92.152                 120       0.55
9     74.125.93.27                 119       0.55

Different IPs: 0
Frequency difference: 0.210

